# A.2 Instatiation/Loading

This notebook demonstrates how we parse and load bibliographic data from the [DBLP dataset](https://dblp.org/) for building our knowledge graph. 

As suggedted in the Assigment, we use the XML version of DBLP, which contains millions of research articles and related entities such as authors, journals, and conferences. This dataset is particularly suited for our assignment, as it provides well-structured information about publications, relationships, and metadata.

In this notebook, we preprocess the data into CSVs using the `XMLToCSV.py` parser, annotate the relationships, and validate the results before loading them into Neo4j.

Finally, we add missing data manually, following the procedure outlined in the PDF.

## Install Required Libraries

We install requirements below.
Note that this does not include the actual tool that we used to extract the XML to hml. This can be found on and https://github.com/ThomHurks/dblp-to-csv.
This git repo just has to be cloned within the current working directory along with the XML *and* .DTD file that we want to process.

In [ ]:
import pandas as pd
!pip install lxml
!pip install -r requirements.txt #this file is included with the DBLP parser tool and includes the necessary libraries

## Parse the DBLP XML into CSVs

After cloning the repo, we can run the parser by running th XMLToCSV.py file. The `--neo4j` flag makes sure the output format is suitable for loading into Neo4j.

The `--relations` parameter lets us define custom relationships between XML tags. For example, it could be simple like the following:

In [ ]:
!python XMLToCSV.py --annotate --neo4j dblp.xml dblp.dtd output.csv --relations author:authored_by journal:published_in

Will create relations for attribute(s): author, journal
Start!
Reading elements from DTD file...
Finding unique attributes for all elements...
Opening output files...
Parsing XML and writing to CSV files...
Writing relation files...
Writing annotated headers...
Generating neo4j-import command...
Writing neo4j-import command to shell script file...
Done after 896.962561 seconds


To get the data as comprehensive as possible, we run the parser with quite some relations. However, not all relations must necessarily be used later since we still have to create a knowledge graph out of it..
In essence, we use the --relations parameter by for example linking the XML Tag to the parent tag.
For example, "--realtions author:authored_by" does the following:

When you find an XML field called author, create a relationship from that value (the Author node) to the parent entity (like a Paper) with the name authored_by.”


In [ ]:
!python XMLToCSV.py --annotate --neo4j dblp.xml dblp.dtd dblp.csv \
--relations \
author:authored_by \
booktitle:presented_at \
journal:published_in \
publisher:published_by


Will create relations for attribute(s): author, booktitle, journal, publisher
Start!
Reading elements from DTD file...
Finding unique attributes for all elements...
Opening output files...
Parsing XML and writing to CSV files...
Writing relation files...
Writing annotated headers...
Generating neo4j-import command...
Writing neo4j-import command to shell script file...
Done after 1110.973302 seconds


## Load Article CSV for Preliminary Exploration

Let's put this into a dataframe to do some preliminary analysis. We will use a filter as we have millions of entries

In [ ]:
import pandas as pd

df = pd.read_csv("dblp_article.csv", sep=';', engine='python')
df.head()


,70072,Kai Schlabitz,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Auswirkung der Digitalisierung auf die Systemlandschaft der Netzbetreiber.,Unnamed: 31,Unnamed: 32,db/journals/ei/ei136.html#Schlabitz19,136,2019
0,70073,Richard Überbacher|Stefan Cecil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,EMF-Personenschutz: Neue Aspekte in der numeri...,NaN,NaN,db/journals/ei/ei137.html#UberbacherC20,137,2020.0
1,70074,Herbert Mang,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Zur Genesis der Forschungsstelle für Integrier...,NaN,NaN,db/journals/ei/ei139.html#Mang22,139,2022.0
2,70075,Christian Diendorfer|Gerwin H. S. Drexler-Schm...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,100 % erneuerbare Energie für Österreichs Indu...,NaN,NaN,db/journals/ei/ei138.html#KnottnerGDD21,138,2021.0
3,70076,Christoph Reichl|Peter Wimberger,NaN,0000-0002-0010-8586|0000-0002-7120-3425,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Application of multilateration for microphone ...,NaN,NaN,db/journals/ei/ei138.html#WimbergerR21,138,2021.0
4,70077,Christian Paul|Fridolin H. Heidler|Wolfgang Sc...,NaN,0000-0002-3440-5431,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Optische Untersuchungen von Blitzeinschlägen i...,NaN,NaN,db/journals/ei/ei139.html#HeidlerPS22,139,2022.0


We firstly notice the following:

- There are quite some NaNs
- There is no header. This is includer in the _header.csv file
- The deliminator is ';' and not ','

This is all usefull to know as it is information that is required when importing into the neo4j database.

## Matching Relationships

To ensure the relationships generated by the parser are valid, we check if the node IDs in the `rel_df` (relationship CSV) actually match those in the author and article CSVs.

We load:
- `dblp_author.csv` — contains author node information
- `dblp_article.csv` — contains article node information
- `dblp_author_authored_by.csv` — contains edges connecting articles to authors

We also use `dblp_article_header.csv` to assign the correct column names

In [ ]:
authors_df = pd.read_csv("dblp_author.csv", sep=";", dtype=str)
with open("dblp_article_header.csv", "r") as f:
    header = f.readline().strip().split(";")

# Load the article data with those column names
articles_df = pd.read_csv("dblp_article.csv", sep=";", names=header, dtype=str)
rel_df = pd.read_csv("dblp_author_authored_by.csv", sep=";", dtype=str)

In [ ]:
authors_df.head()

,:ID,author:string
0,11554227,Arnon Rosenthal
1,11554228,Robin Cover
2,11554229,Alin Deutsch
3,11554230,Mary F. Fernandez
4,11554231,Daniela Florescu


In [ ]:
rel_df.head()

,:START_ID,:END_ID
0,1,11554227
1,3759619,11554227
2,8015887,11554227
3,9799700,11554227
4,7812123,11554227


In [ ]:
articles_df.head()

,article:ID,author:string[],author-aux:string,author-orcid:string[],booktitle:string,cdate:date,cdrom:string,cite:string[],cite-label:string[],crossref:string,...,publtype:string,stream:string,sub:string[],sup:string[],title:string,title-bibtex:string,tt:string[],url:string[],volume:string,year:int
0,70072,Kai Schlabitz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Auswirkung der Digitalisierung auf die Systeml...,NaN,NaN,db/journals/ei/ei136.html#Schlabitz19,136,2019
1,70073,Richard Überbacher|Stefan Cecil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,EMF-Personenschutz: Neue Aspekte in der numeri...,NaN,NaN,db/journals/ei/ei137.html#UberbacherC20,137,2020
2,70074,Herbert Mang,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Zur Genesis der Forschungsstelle für Integrier...,NaN,NaN,db/journals/ei/ei139.html#Mang22,139,2022
3,70075,Christian Diendorfer|Gerwin H. S. Drexler-Schm...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,100 % erneuerbare Energie für Österreichs Indu...,NaN,NaN,db/journals/ei/ei138.html#KnottnerGDD21,138,2021
4,70076,Christoph Reichl|Peter Wimberger,NaN,0000-0002-0010-8586|0000-0002-7120-3425,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Application of multilateration for microphone ...,NaN,NaN,db/journals/ei/ei138.html#WimbergerR21,138,2021


## Validate ID Matching Between Nodes and Relationships

We now check how many of the relationship IDs exist in the node tables. This ensures that both `:START_ID` (paper) and `:END_ID` (author) point to actual existing nodes.


In [ ]:

# Check overlap
print("START_ID match:", rel_df[":START_ID"].isin(authors_df[":ID"]).sum())
print("END_ID match:", rel_df[":END_ID"].isin(articles_df["article:ID"]).sum())

# Check overlap
print("START_ID match:", rel_df[":START_ID"].isin(articles_df["article:ID"]).sum())
print("END_ID match:", rel_df[":END_ID"].isin(authors_df[":ID"]).sum())


START_ID match: 0
END_ID match: 0
START_ID match: 13080262
END_ID match: 29763959


## Cleaning the cite Relationships

The raw `cite` relationships extracted from DBLP often apparently contained malformed data. Importing it always gave us an error, so we clean the `dblp_cite_has_citation.csv` file:

We'll remove any rows with missing IDs or references to unknown article IDs.

This yields: `dblp_cite_has_citation_cleaned.csv` which will be used during the import.


In [ ]:
import pandas as pd
# Load original citation relationships
cite_df = pd.read_csv("dblp_cite_has_citation.csv", sep=";", dtype=str)

# Drop rows with missing START or END IDs
cite_df = cite_df.dropna(subset=[":START_ID", ":END_ID"])

# Optionally, check if cited articles exist in your article dataset
with open("dblp_article_header.csv", "r") as f:
    header = f.readline().strip().split(";")


valid_articles = set(pd.read_csv("dblp_article.csv", sep=";", names=header, dtype=str)["article:ID"])
cite_df = cite_df[cite_df[":START_ID"].isin(valid_articles) & cite_df[":END_ID"].isin(valid_articles)]

# Save cleaned version
cite_df.to_csv("dblp_cite_has_citation_cleaned.csv", sep=";", index=False)

print(f"Cleaned cite relations saved: {cite_df.shape[0]} rows")


Cleaned cite relations saved: 0 rows


Note that this dataset will be filtered to 0 rows, meaning there will not be any actuall data therer. We can thus disregard has citation when importing. We will add them later manually

## Bulk Import into Neo4j

Once the CSVs are ready, we load the data into Neo4j using the `neo4j-admin database import` tool.

We did this the following way:

1. Stop the running Neo4j server (if it's running).
2. Move all CSV files into the `import` directory inside the Neo4j folder.
3. Run the following command from the root of your Neo4j installation folder.

This command creates a new database using all the parsed CSVs from the DBLP dataset and sets up the nodes and relationships correctly.


```bash
./bin/neo4j-admin database import full \
  --delimiter=";" \
  --array-delimiter="|" \
  --id-type=string \
  --overwrite-destination=true \
  --verbose \
  --skip-bad-relationships=true \
  --bad-tolerance=3500 \
  --nodes=article=import/dblp_article_header.csv,import/dblp_article.csv \
  --nodes=mastersthesis=import/dblp_mastersthesis_header.csv,import/dblp_mastersthesis.csv \
  --nodes=phdthesis=import/dblp_phdthesis_header.csv,import/dblp_phdthesis.csv \
  --nodes=proceedings=import/dblp_proceedings_header.csv,import/dblp_proceedings.csv \
  --nodes=www=import/dblp_www_header.csv,import/dblp_www.csv \
  --nodes=incollection=import/dblp_incollection_header.csv,import/dblp_incollection.csv \
  --nodes=book=import/dblp_book_header.csv,import/dblp_book.csv \
  --nodes=data=import/dblp_data_header.csv,import/dblp_data.csv \
  --nodes=inproceedings=import/dblp_inproceedings_header.csv,import/dblp_inproceedings.csv \
  --nodes=author=import/dblp_author.csv \
  --relationships=authored_by=import/dblp_author_authored_by.csv \
  --nodes=booktitle=import/dblp_booktitle.csv \
  --relationships=presented_at=import/dblp_booktitle_presented_at.csv \
  --nodes=journal=import/dblp_journal.csv \
  --relationships=published_in=import/dblp_journal_published_in.csv \
  --nodes=publisher=import/dblp_publisher.csv \
  --relationships=published_by=import/dblp_publisher_published_by.csv



This will give us a graph like the following. Note that **article, phdthesis, incollection, mastersthesis, book, inproceeding** are all kinds of papers as described in the pdf report. We could not filter for only articles so we decided to include all and model it as paper in the conceptual model!

In [ ]:
from IPython.display import HTML
def display_image_with_white_background(img_path): #this function avoids bad looking images and caching problems when using html in jupyter notebook
    html = f"""
    <div style="background-color: white; padding: 10px; display: inline-block; border: 1px solid #ddd; border-radius: 8px;">
      <img src="{img_path}" alt="Graph" />
    </div>
    """
    return HTML(html)

display_image_with_white_background("graph.png")

Furthermore, it was impossible for us to remove the www node so we decided to just hide it when displaying the schema. Including the www would lead to what we have below.
Other than that specific node, we can notice that it looks very similiar to what we have in our comceptual graph as seen in the report.
Summarizing the different kinds of paper, we have something like:


In [4]:

display_image_with_white_background("graph2.png")

## Inserting Data Manually

To adress the missing gaps and to full address the requirement in the assigment, we used these Cypher queries to add the missing data:


### Abstract

```bash
UNWIND [
  {
    key: "journals/ral/AhnSD18",
    abstract: "This editorial introduces recent trends in autonomous agricultural robotics, summarizing technological advancements that enable precision farming and environmental sustainability."
  },
  {
    key: "journals/ral/YoonJPGSSMC25",
    abstract: "We propose a three-finger adaptive gripper with integrated suction cups that enhances grasping capability for objects with varying surfaces, enabling robust manipulation in unstructured environments."
  },
  {
    key: "journals/ral/RoferBBVB22",
    abstract: "This paper introduces Kineverse, a symbolic articulation framework that supports model-agnostic planning for mobile manipulation, enabling flexible robot behavior across diverse platforms."
  },
  {
    key: "journals/ral/ZhangYTXW22a",
    abstract: "This note provides corrections to the MR-TopoMap system for multi-robot exploration, addressing previous limitations and improving navigation efficiency in communication-restricted environments."
  },
  {
    key: "journals/ral/LiuJZM23",
    abstract: "We introduce a collision-free motion generation framework using stochastic optimization combined with signed distance field networks for articulated robot path planning in complex scenarios."
  },
  {
    key: "journals/ral/IshiMI20",
    abstract: "This study examines how android robots can convey social attitudes like politeness and friendliness through variations in pointing gestures, contributing to more natural human-robot interactions."
  },
  {
    key: "journals/ral/LiZLWCTL24",
    abstract: "DBPF is a novel framework for dynamic bin-picking that improves robotic perception and grasping efficiency in real-time environments through adaptive planning and robust object detection."
  },
  {
    key: "journals/ral/SaxenaGJ24",
    abstract: "We present a reward shaping strategy using funnel functions to guide reinforcement learning agents in satisfying Signal Temporal Logic specifications during complex robotic tasks."
  },
  {
    key: "journals/ral/KimKP19",
    abstract: "A soft-actuated tripod robot is introduced, utilizing membrane vibration actuators for locomotion, offering a simple yet effective design for mobile platforms with minimal components."
  },
  {
    key: "journals/ral/FariasMSB21",
    abstract: "This paper presents a method for simultaneous tactile exploration and grasp refinement, enabling robots to improve manipulation strategies through continuous sensor feedback on unknown objects."
  }
] AS row
MATCH (a:article {key: row.key})
SET a.abstract = row.abstract;


### City and Edition

We manually add to selected proceedings nodes:  location (city) and edition number (edition):

```bash

UNWIND [
  {key: "conf/stacs/1990", city: "Rouen", edition: "7"},
  {key: "conf/icml/2005", city: "Bonn", edition: "22"},
  {key: "conf/kdd/2011", city: "San Diego", edition: "17"}
] AS confData
MATCH (p:proceedings {key: confData.key})
SET p.city = confData.city,
    p.edition = confData.edition
RETURN p.key, p.city, p.edition;

### Keywords

This adds information to papers through a new Keyword node and the HAS_KEYWORD relationship.

```bash
UNWIND [
  {paperKey: "journals/pr/NaseemTB12", keyword: "face recognition"},
  {paperKey: "journals/pr/NaseemTB12", keyword: "robust regression"},
  {paperKey: "journals/pr/NaseemTB12", keyword: "pattern recognition"},
  {paperKey: "conf/icml/2005/SomeAuthor", keyword: "machine learning"}
] AS kwData

MATCH (p {key: kwData.paperKey})
MERGE (k:Keyword {text: kwData.keyword})
MERGE (p)-[:HAS_KEYWORD]->(k);




```bash

MATCH (a:article)-[h:HAS_KEYWORD]-(k:Keyword) RETURN a, h, k


### Citation

We manually create citaiton links has the included has_citation did not work


```bash
UNWIND [
  {source: "journals/pr/NaseemTB12", target: "journals/ml/Bishop06"},
  {source: "journals/pr/NaseemTB12", target: "journals/pr/ChenB99"},
  {source: "journals/pr/NaseemTB12", target: "journals/pr/HuangJZ08"},
  // HanY10b is cited in three others

  {source: "journals/jicce/HanY10b", target: "journals/jicce/Lim11b"},
  {source: "journals/jicce/HanY10b", target: "journals/jicce/Kim09g"},
  {source: "journals/jicce/HanY10b", target: "journals/jicce/PhamMLY08"},
   // Lim11b is cited in two others

  {source: "journals/jicce/Lim11b", target: "journals/jicce/Oh12"},
  {source: "journals/jicce/Lim11b", target: "journals/jicce/Kim09g"},
   // KimYS04 is cited in one other

  {source: "journals/jicce/KimYS04", target: "journals/jicce/LeeKLK10"}
] AS row
MERGE (src:article {key: row.source})
MERGE (tgt:article {key: row.target})
MERGE (src)-[:HAS_CITATION]->(tgt);


```bash
MATCH (a)-[r:HAS_CITATION]->(b)
RETURN a.title AS citing, type(r) AS relation, b.title AS cited
LIMIT 10;

### Reviewed by

We create a custom relationship REVIEWED_BY between articles and authors who reviewed them:


```bash

UNWIND [
  {paperKey: "journals/pr/NaseemTB12", reviewerName: "Guangyi Chen 0001"},
  {paperKey: "journals/pr/NaseemTB12", reviewerName: "Anil K. Jain 0001"},
  {paperKey: "journals/pr/JainGC02", reviewerName: "Roberto Togneri"},
  {paperKey: "journals/pr/JainGC02", reviewerName: "Imran Naseem"},
  {paperKey: "journals/pr/ChenB99", reviewerName: "David Zhang 0001"}
] AS reviewData

MATCH (p:article {key: reviewData.paperKey})
MATCH (a:author {author: reviewData.reviewerName})
MERGE (p)-[:REVIEWED_BY]->(a);


```bash

MATCH (a:article)-[r:REVIEWED_BY]-(p:author) RETURN a, r, p

### Conference Edition

```bash
CREATE (ce:ConferenceEdition {name: 'Fifth Workshop on Computer Architecture for Non-Numeric Processing', year: 1980,month:'April',  venue: 'Pacific Grove', editionID:"conf/sigmod/80nnp"})

CREATE (c:Conference {  name: 'Computer Architecture'})


```` bash
CREATE CONSTRAINT ON (ce:ConferenceEdition)
ASSERT ce.editionID IS UNIQUE;

```bash
match (ce:ConferenceEdition {editionID:"conf/sigmod/80nnp"})
match (c:Conference {name:"Computer Architecture"})
merge (ce)-[:part_of] ->(c)

``` BASH
MATCH (ce:ConferenceEdition {editionID:"conf/sigmod/80nnp"})
MATCH (p:proceedings {key:"conf/sigmod/80nnp"})
MERGE (p)-[:of_edition] ->(ce)

### Additional Data for Queries

To get meaningful results in the queries of the next tasks, we add some more artificial data. This is done in such a way that everything is connected, creating richly connected subgraph for those articles, enabling queries like:

    “What articles were presented at the 1980 SIGMOD conference?”

    “Which city hosted edition X of conference Y?”

    “What is the h-index of authors who presented papers at SIGMOD 1980?”


``` BASH

UNWIND [
  {articleKey:"journals/jicce/HanY10b"},
  {articleKey:"journals/jicce/Lim11b"},
  {articleKey:"journals/jicce/KimYS04"}
] AS row

// 1) Find the article, proceedings, edition and conference
MATCH (a:article {key: row.articleKey})
MATCH (pr:proceedings {key: "conf/sigmod/80nnp"})
MATCH (ed:ConferenceEdition {editionID: "conf/sigmod/80nnp"})
MATCH (c:Conference {name: "Computer Architecture"})

// 2) Create or find a single booktitle for this edition
MERGE (bt:booktitle {
  title: "SIGMOD 1980 Proceedings"    // label it however makes sense
})

// 3) Link everything up
MERGE (a)-[:PRESENTED_AT]->(bt)
MERGE (pr)-[:PRESENTED_AT]->(bt)
MERGE (bt)-[:OF_EDITION]->(ed)
MERGE (ed)-[:PART_OF]->(c);

## Updated Graph

After running these queries and hiding www, the graph looks like the following. We can see how this matches the conceptual design!

In [ ]:
display_image_with_white_background("graph3.png")